In [41]:
import psycopg2

PORT = 54320

# connect to the postgresql db
conn = psycopg2.connect(
  host="localhost",
  port=PORT,
  dbname="coolseel",
  user="coolseel",
  password="password"
)

cur = conn.cursor()

In [39]:
# SELECT streamer.name, COUNT(stream) COUNT(segment), COUNT(message)
# FROM streamer
# LEFT JOIN stream ON streamer.id = stream.streamer_id
# LEFT JOIN segment ON stream.id = segment.stream_id
# LEFT JOIN message ON segment.id = message.segment_id
# GROUP BY streamer.name
# ORDER BY COUNT(stream) DESC

cur.execute("""
SELECT "Streamer".name, COUNT(distinct "Stream".id), COUNT(distinct "Segment".id), COUNT(distinct "Message".id)
FROM "Streamer"
LEFT JOIN "Stream" ON "Streamer".id = "Stream"."streamerId"
LEFT JOIN "Segment" ON "Stream".id = "Segment"."streamId"
LEFT JOIN "Message" ON "Segment".id = "Message"."segmentId"
WHERE "Stream".live = true
GROUP BY "Streamer".name
ORDER BY COUNT("Stream".id) DESC
""")
rows = cur.fetchall()
print("Streamer, Stream Count, Segment Count, Message Count")
for row in rows:
  print(row)

Streamer, Stream Count, Segment Count, Message Count
('northernlion', 2, 2, 62011)
('boxbox', 1, 2, 1851)


In [32]:
conn.rollback()

In [42]:
# Select all segements associated with streamer 1
cur.execute("""
SELECT "Segment".id, "Segment".start, "Segment".duration, "Segment".url
FROM "Segment"
LEFT JOIN "Stream" ON "Segment"."streamId" = "Stream".id
WHERE "Stream"."streamerId" = 1
""")
rows = cur.fetchall()
print("Segment ID, Start, End, Duration, URL")
for row in rows:
  print(row)

UndefinedColumn: column Segment.end does not exist
LINE 2: SELECT "Segment".id, "Segment".start, "Segment".end, "Segmen...
                                              ^


In [34]:
# insert 3 fake streams with 100000 messages each for streamer 1
for i in range(3):
  cur.execute("""
  INSERT INTO "Stream" ("streamerId", "live", "start")
  VALUES (1, true, NOW())
  RETURNING id
  """)
  stream_id = cur.fetchone()[0]
  cur.execute("""
  INSERT INTO "Segment" ("streamId", game, start)
  VALUES (%s, %s, NOW())
  RETURNING id
  """, (stream_id, "Test Game"))
  segment_id = cur.fetchone()[0]
  for j in range(100000):
    cur.execute("""
    INSERT INTO "Message" ("segmentId", "secondsSinceStart", "text", "username")
    VALUES (%s, %s, %s, %s)
    """, (segment_id, j, f"This is a test message number {j}", "test_user"))

In [37]:
# delete all messages from "test_user"
cur.execute("""
DELETE FROM "Message"
WHERE "username" = 'test_user'
""")
# delete all segments with no messages
cur.execute("""
DELETE FROM "Segment"
WHERE NOT EXISTS (
  SELECT 1
  FROM "Message"
  WHERE "Message"."segmentId" = "Segment".id
)
""")
# delete all streams with no segments
cur.execute("""
DELETE FROM "Stream"
WHERE NOT EXISTS (
  SELECT 1
  FROM "Segment"
  WHERE "Segment"."streamId" = "Stream".id
)
""")

In [38]:
conn.commit()